<a href="https://colab.research.google.com/github/pskshyam/SemanticSegmentation_KittiRoad/blob/master/SemanticSegmentation_KittyRoadDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

Let’s walk through a step-by-step implementation of the most popular architecture for semantic segmentation — the Fully-Convolutional Net (FCN). We’ll implement it using the TensorFlow library in Python 3, along with other dependencies such as Numpy and Scipy.

In this exercise, we will label the pixels of road in images using FCN. We’ll work with the Kitti Road Dataset for road/lane detection.

## Key Features of FCN Architecture


* FCN transfers knowledge from VGG16 to perform semantic segmentation.
* The fully connected layers of VGG16 is converted to fully convolutional layers, using 1x1 convolution. This process produces a class presence heat map in low resolution.
* The upsampling of these low resolution semantic feature maps is done using transposed convolutions (initialized with bilinear interpolation filters).
* At each stage, the upsampling process is further refined by adding features from coarser but higher resolution feature maps from lower layers in VGG16.
* Skip connection is introduced after each convolution block to enable the subsequent block to extract more abstract, class-salient features from the previously pooled features.


## Steps involved in FCN-8 Implementation

There are 3 versions of FCN (FCN-32, FCN-16, FCN-8). We’ll implement FCN-8, as detailed step-by-step below:

* Encoder: A pre-trained VGG16 is used as an encoder. The decoder starts from Layer 7 of VGG16.
* FCN Layer-8: The last fully connected layer of VGG16 is replaced by a 1x1 convolution.
* FCN Layer-9: FCN Layer-8 is upsampled 2 times to match dimensions with Layer 4 of VGG 16, using transposed convolution with parameters: (kernel=(4,4), stride=(2,2), paddding=’same’). After that, a skip connection was added between Layer 4 of VGG16 and FCN Layer-9.
* FCN Layer-10: FCN Layer-9 is upsampled 2 times to match dimensions with Layer 3 of VGG16, using transposed convolution with parameters: (kernel=(4,4), stride=(2,2), paddding=’same’). After that, a skip connection was added between Layer 3 of VGG 16 and FCN Layer-10.
* FCN Layer-11: FCN Layer-10 is upsampled 4 times to match dimensions with input image size so we get the actual image back and depth is equal to number of classes, using transposed convolution with parameters:(kernel=(16,16), stride=(8,8), paddding=’same’).

## Steps involved in Semantic Segmentation

* Training Parameters

   289 images used in training

   289 labels used in training
   
   290 images used in testing
   
   Learning rate = 0.0001

   Epochs = 30

   Samples per epoch = 100

   Batch Size = 8

In [0]:
#Import necessary libraries
import os
import argparse
import warnings
import tensorflow as tf
import helper
from helper import *
#from helper import gen_batch_function, save_inference_samples
from distutils.version import LooseVersion
from os.path import join, expanduser
import project_tests as tests
import image_augmentation
from image_augmentation import perform_augmentation

In [22]:
#Download vgg trained model and Kitti Road Dataset
!wget https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip

--2019-03-01 14:39:27--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.72.8
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.72.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470992343 (449M) [application/zip]
Saving to: ‘data_road.zip’

data_road.zip       100%[===================>] 449.17M  42.4MB/s    in 10s     

2019-03-01 14:39:38 (43.3 MB/s) - ‘data_road.zip’ saved [470992343/470992343]



In [45]:
#Upload python files

from google.colab import files

uploaded = files.upload()

Saving main.py to main.py


In [52]:
!ls -ltr data

total 8
drwxrwxr-x 4 root root 4096 Feb  9  2015 data_road
drwxr-xr-x 3 root root 4096 May 31  2017 vgg


In [53]:
#Run the main program in main.py
!python main.py

TensorFlow Version: 1.13.1
2019-03-01 14:51:02.871528: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-01 14:51:02.871768: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x14a91e0 executing computations on platform Host. Devices:
2019-03-01 14:51:02.871816: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-01 14:51:02.938600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-01 14:51:02.939188: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x14a9b80 executing computations on platform CUDA. Devices:
2019-03-01 14:51:02.939292: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-03-01 14:51:02.939685: I tensorflow/core/common_runtime/gpu/

## Reference
https://github.com/ndrplz/self-driving-car/tree/master/project_12_road_segmentation